## Pandas analysis

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a couple of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns (a BX counter is made of 30 TDC counters), and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [51]:
# If you didn't download it yet, please get the relevant file now!
!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ~/data/

--2024-11-28 10:02:38--  https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt
Risoluzione di www.dropbox.com (www.dropbox.com)... 162.125.69.18, 2620:100:6025:18::a27d:4512
Connessione a www.dropbox.com (www.dropbox.com)|162.125.69.18|:443... connesso.
Richiesta HTTP inviata, in attesa di risposta... 302 Found
Posizione: https://www.dropbox.com/scl/fi/pkkpoxlm7beasryexpdf8/data_000637.txt?rlkey=rkm2em1v57hewglzelmin21c9 [segue]
--2024-11-28 10:02:38--  https://www.dropbox.com/scl/fi/pkkpoxlm7beasryexpdf8/data_000637.txt?rlkey=rkm2em1v57hewglzelmin21c9
Riutilizzo della connessione esistente a www.dropbox.com:443.
Richiesta HTTP inviata, in attesa di risposta... 302 Found
Posizione: https://uc468f273c4133ac26a7a00b7c40.dl.dropboxusercontent.com/cd/0/inline/CfNFUABuEcIv3u5uTkWMDp2RSPu2YuvwIDMbBgyys3qy2lXOuouE0YWhDvoDubd3j_k_khn7QONuwtBBCbcSJ7VmIMoMxTCi21FFJuHZp8-CPkjRI56XYyw9ahFagFMSLjM/file# [segue]
--2024-11-28 10:02:39--  https://uc468f273c4133ac26a7a00b7c40.dl.dropboxusercontent.

1\. Create a Pandas DataFrame reading N rows of the 'data_000637.txt' dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k.

2\. Find out the number of BX in a ORBIT (the value 'x').

3\. Find out how much the data taking lasted. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading out the whole dataset.

4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information).

5\. Replace the values (all 1) of the HEAD column randomly with 0 or 1.

6\. Create a new DataFrame that contains only the rows with HEAD=1.

7\. Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel

8\. Use the groupby method to find out the noisy channels, i.e. the TDC channels with most counts (say the top 3)

9\. Count the number of unique orbits. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139

In [52]:
import pandas as pd

In [53]:
#1
file_name = "/Users/raffaele/data/data_000637.txt"
data = pd.read_csv(file_name)
rows = int(5E5)
dataset = data[0: rows]
dataset

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
499995,1,1,25,3869205221,2330,28
499996,1,1,23,3869205221,2335,10
499997,1,1,139,3869205221,2336,0
499998,1,1,24,3869205221,2333,21


In [54]:
#2
bx_counters = dataset["BX_COUNTER"]
orbit_time = max(bx_counters) + 1
orbit_time #in ns

3564

In [55]:
#3
times = dataset["ORBIT_CNT"]*orbit_time*25 + dataset["BX_COUNTER"]*25 + dataset["TDC_MEAS"]*25/30
period = max(times) - min(times)
period

450310428.3125

In [56]:
#4
timed_dataframe = pd.concat([dataset, times], axis = 1)
timed_dataframe

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,0
0,1,0,123,3869200167,2374,26,3.447457e+14
1,1,0,124,3869200167,2374,27,3.447457e+14
2,1,0,63,3869200167,2553,28,3.447457e+14
3,1,0,64,3869200167,2558,19,3.447457e+14
4,1,0,64,3869200167,2760,25,3.447457e+14
...,...,...,...,...,...,...,...
499995,1,1,25,3869205221,2330,28,3.447462e+14
499996,1,1,23,3869205221,2335,10,3.447462e+14
499997,1,1,139,3869205221,2336,0,3.447462e+14
499998,1,1,24,3869205221,2333,21,3.447462e+14


In [57]:
import numpy as np
list = np.random.randint(2, size = rows)
timed_dataframe["HEAD"] = list
timed_dataframe


,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,0
0,1,0,123,3869200167,2374,26,3.447457e+14
1,1,0,124,3869200167,2374,27,3.447457e+14
2,0,0,63,3869200167,2553,28,3.447457e+14
3,0,0,64,3869200167,2558,19,3.447457e+14
4,0,0,64,3869200167,2760,25,3.447457e+14
...,...,...,...,...,...,...,...
499995,0,1,25,3869205221,2330,28,3.447462e+14
499996,0,1,23,3869205221,2335,10,3.447462e+14
499997,0,1,139,3869205221,2336,0,3.447462e+14
499998,0,1,24,3869205221,2333,21,3.447462e+14
